In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()

        self.enc1 = self.double_conv(in_channels, 64)
        self.enc2 = self.double_conv(64, 128)
        self.enc3 = self.double_conv(128, 256)
        self.enc4 = self.double_conv(256, 512)
        self.enc5 = self.double_conv(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.up4 = self.upconv(1024, 512)
        self.dec4 = self.double_conv(1024, 512)

        self.up3 = self.upconv(512, 256)
        self.dec3 = self.double_conv(512, 256)

        self.up2 = self.upconv(256, 128)
        self.dec2 = self.double_conv(256, 128)

        self.up1 = self.upconv(128, 64)
        self.dec1 = self.double_conv(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def double_conv(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def upconv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):

        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))

        x = self.up4(enc5)
        x = torch.cat([x, enc4], dim=1)
        x = self.dec4(x)

        x = self.up3(x)
        x = torch.cat([x, enc3], dim=1)
        x = self.dec3(x)

        x = self.up2(x)
        x = torch.cat([x, enc2], dim=1)
        x = self.dec2(x)

        x = self.up1(x)
        x = torch.cat([x, enc1], dim=1)
        x = self.dec1(x)

        return self.final_conv(x)



In [ ]:
import os
import zipfile
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import torch
from torch import nn

# Function to extract images from the ZIP file
def extract_images_from_zip(zip_path, folder_name="MSFD/2/img", target_size=(256, 256)):
    print("Entered ZIP")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Iterate over the files inside folder '2/img'
        print("Entered folder")
        for file_name in zip_ref.namelist():
            if file_name.startswith(folder_name) and file_name.endswith(('.png', '.jpg', '.jpeg')):
                print(f"Processing file: {file_name}")
                with zip_ref.open(file_name) as file:
                    img = Image.open(file)
                    img = img.resize(target_size)  # Resize the image to a consistent size
                    yield np.array(img)  # Yield the resized image as a numpy array

def batch_generator(zip_path, batch_size=200, target_size=(256, 256)):
    image_count = 0
    batch = []
    print("Entered batch")
    for img in extract_images_from_zip(zip_path, target_size=target_size):
        batch.append(img)  # Add the resized image to the batch
        image_count += 1
        if image_count == batch_size:
            yield np.array(batch)  # Return a batch of images
            batch = []  # Reset batch
            image_count = 0

def unet_pipeline(zip_path):
    # Initialize the UNet model
    model = UNet(in_channels=3, out_channels=1)  # Adjust in_channels if needed (e.g., 3 for RGB images)
    model.eval()  # Set the model to evaluation mode (or model.train() for training)

    # Transfer the model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Process batches and pass them to the model
    for image_batch in batch_generator(zip_path, batch_size=1000):
        image_batch_tensor = torch.tensor(image_batch).float().to(device)  # Convert batch to tensor
        image_batch_tensor = image_batch_tensor.permute(0, 3, 1, 2)  # Change dimensions to [batch, channels, height, width]
        
        # Forward pass through UNet
        with torch.no_grad():
            output = model(image_batch_tensor)
        
        print(f"Processed a batch of {len(image_batch)} images.")
        # Here, you can save the output or process it further

# # Upload ZIP file to Colab
# uploaded = files.upload()  # This will open a file picker to upload your ZIP file

# The user uploads the file, let's assume the file is 'MSFD.zip'
zip_path = "C:/Users/cdate/Documents/Chirag/IIITB/SEM2/VR/MiniProject/MSFD.zip"  # Path to the uploaded ZIP file in Colab

# Run the UNet pipeline
unet_pipeline(zip_path)
